In [1]:
import os
import time
from colorutils import Color
from colorutils import ArithmeticModel
import open3d as o3d
from open3d import visualization
import cv2
import numpy as np
from pathlib import Path

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
JOINT_MAP = {
    'Hips': 0,
    'LeftUpLeg': 1,
    'RightUpLeg': 2,
    'Spine': 3,
    'LeftLeg': 4,
    'RightLeg': 5,
    'Spine1': 6,
    'LeftFoot': 7,
    'RightFoot': 8,
    'Spine2': 9,
    'LeftToeBase': 10,
    'RightToeBase': 11,
    'Neck': 12,
    'LeftShoulder': 13,
    'RightShoulder': 14,
    'Head': 15,
    'LeftArm': 16,
    'RightArm': 17,
    'LeftForeArm': 18,
    'RightForeArm': 19,
    'LeftHand': 20,
    'RightHand': 21,
}
JOINT_PAIRS = [
    # ['Hips', 'Spine'],
    # ['Spine', 'Spine1'],
    # ['Spine1', 'Spine2'],
    ['Spine2', 'Neck'],
    ['Neck', 'Head'],
    ['Spine2', 'RightArm'],
    # ['RightShoulder', 'RightArm'],
    ['RightArm', 'RightForeArm'],
    ['RightForeArm', 'RightHand'],
    ['Spine2', 'RightUpLeg'],
    ['RightUpLeg', 'RightLeg'],
    ['RightLeg', 'RightFoot'],
    # ['RightFoot', 'RightToeBase'],
    ['Spine2', 'LeftArm'],
    # ['LeftShoulder', 'LeftArm'],
    ['LeftArm', 'LeftForeArm'],
    ['LeftForeArm', 'LeftHand'],
    ['Spine2', 'LeftUpLeg'],
    ['LeftUpLeg', 'LeftLeg'],
    ['LeftLeg', 'LeftFoot'],
    # ['LeftFoot', 'LeftToeBase']
]

In [3]:
color_groups = {
    'brown_rgb': [(210,180,140),(222,184,135),(205,133,63),(210,105,30),(139,69,19),(160,82,45)],
    'red_rgb': [(240,128,128),(250,128,114),(205,92,92),(220,20,60)],
    'green_rgb': [(0,255,0),(50,205,50),(34,139,34),(0,128,0)],
    'orange_rgb': [(255,165,0),(255,140,0),(255,99,71),(255,69,0)],
    'blue_rgb': [(173,216,230),(135,206,250),(135,206,235),(0,191,255)],
}
joint_groups = {
    'center_joints': ['Hips','Spine','Spine1','Spine2','Neck','Head'],
    'right_upper_joints': ['RightShoulder','RightArm','RightForeArm','RightHand'],
    'right_lower_joints': ['RightUpLeg','RightLeg','RightFoot','RightToeBase'],
    'left_upper_joints': ['LeftShoulder','LeftArm','LeftForeArm','LeftHand'],
    'left_lower_joints': ['LeftUpLeg','LeftLeg','LeftFoot','LeftToeBase'],
}
color_joint_group_pair = {
    'brown_rgb': 'center_joints',
    'red_rgb': 'right_upper_joints',
    'green_rgb': 'right_lower_joints',
    'orange_rgb': 'left_upper_joints',
    'blue_rgb': 'left_lower_joints',
}

In [4]:
color_joint_pair = dict()

for color_group_name, joint_group_name in color_joint_group_pair.items():
    color_list = color_groups[color_group_name]
    joint_list = joint_groups[joint_group_name]
    
    if len(color_list) < len(joint_list):
        print(f"{color_group_name} color list shorter than number of joints in {joint_group_name}")
    else:
        for idx in range(len(color_list)):
            color_joint_pair[joint_list[idx]] = color_list[idx]

In [5]:
color_joint_pair

{'Hips': (210, 180, 140),
 'Spine': (222, 184, 135),
 'Spine1': (205, 133, 63),
 'Spine2': (210, 105, 30),
 'Neck': (139, 69, 19),
 'Head': (160, 82, 45),
 'RightShoulder': (240, 128, 128),
 'RightArm': (250, 128, 114),
 'RightForeArm': (205, 92, 92),
 'RightHand': (220, 20, 60),
 'RightUpLeg': (0, 255, 0),
 'RightLeg': (50, 205, 50),
 'RightFoot': (34, 139, 34),
 'RightToeBase': (0, 128, 0),
 'LeftShoulder': (255, 165, 0),
 'LeftArm': (255, 140, 0),
 'LeftForeArm': (255, 99, 71),
 'LeftHand': (255, 69, 0),
 'LeftUpLeg': (173, 216, 230),
 'LeftLeg': (135, 206, 250),
 'LeftFoot': (135, 206, 235),
 'LeftToeBase': (0, 191, 255)}

In [6]:
point_colors = []

for joint_name in JOINT_MAP.keys():
    point_colors.append(color_joint_pair[joint_name])
    
point_colors = np.asarray(point_colors)
point_colors = np.divide(point_colors, 255)

In [7]:
point_pairs = []
line_colors = []

for pairs in JOINT_PAIRS:
    point_pairs.append((JOINT_MAP[pairs[0]], JOINT_MAP[pairs[1]]))
    first_point_c = Color(color_joint_pair[pairs[0]], arithmetic=ArithmeticModel.BLEND)
    second_point_c = Color(color_joint_pair[pairs[1]], arithmetic=ArithmeticModel.BLEND)
    line_colors.append(tuple(first_point_c+second_point_c))
        
point_pairs = np.asarray(point_pairs)
line_colors = np.divide(np.asarray(line_colors), 255)

In [8]:
mdm_result_path = Path("/home/notingcode/Projects/3d/")

npy_filepaths = mdm_result_path.glob("*.npy")

all_motion_data = dict()

for npy_fp in npy_filepaths:
    motion_data = np.load(npy_fp, allow_pickle=True)
    
    for key, val in motion_data.item().items():
        if all_motion_data.get(key, None) is None:
            all_motion_data[key] = list()
        
        all_motion_data[key].append(val)
        
        print(f"key: {key} | val type: {type(val)}")
    
    print(f"loaded \n{npy_fp}")

key: motion | val type: <class 'numpy.ndarray'>
key: text | val type: <class 'list'>
key: lengths | val type: <class 'numpy.ndarray'>
key: num_samples | val type: <class 'int'>
key: num_repetitions | val type: <class 'int'>
loaded 
/home/notingcode/Projects/3d/results.npy


In [9]:
def adjust_neck(motion):
    left_shoulder = motion[:,JOINT_MAP['LeftShoulder'],:,:]
    right_shoulder = motion[:,JOINT_MAP['RightShoulder'],:,:]
    spine2 = motion[:,JOINT_MAP['Spine2'],:,:]
    
    motion[:,JOINT_MAP['Spine2'],:,:] = ((left_shoulder+right_shoulder)*0.45 + spine2*0.1)

In [10]:
def adjust_uplegs2hips(motion):
    left_upleg = motion[:,JOINT_MAP['LeftUpLeg'],:,:]
    right_upleg = motion[:,JOINT_MAP['RightUpLeg'],:,:]
    
    right_hip = (left_upleg*-0.1+right_upleg*1.1)
    left_hip = (left_upleg*1.1+right_upleg*-0.1)
    
    motion[:,JOINT_MAP['LeftUpLeg'],:,:] = left_hip
    motion[:,JOINT_MAP['RightUpLeg'],:,:] = right_hip

In [11]:
# Non-blocking visualization implementation
vis1 = visualization.Visualizer()
vis1.create_window(left=0,top=0)
vc1 = vis1.get_view_control()

render_option1 = vis1.get_render_option()
render_option1.background_color = np.asarray([0,0,0])

pcd = o3d.geometry.PointCloud()
lineSet = o3d.geometry.LineSet()

custom_motion_data = all_motion_data['motion'][0]

# fix points
adjust_neck(custom_motion_data)
adjust_uplegs2hips(custom_motion_data)

for action_idx in range(all_motion_data['motion'][0].shape[0]//3):

    entire_motion = custom_motion_data[action_idx,:,:,:]

    for frame_idx in range(entire_motion.shape[2]//3):
        
        pcd.clear()
        lineSet.clear()
        pcd.points = o3d.utility.Vector3dVector(entire_motion[:,:,frame_idx])
        pcd.colors = o3d.utility.Vector3dVector(point_colors)
        lineSet.points = o3d.utility.Vector3dVector(entire_motion[:,:,frame_idx])
        lineSet.lines = o3d.utility.Vector2iVector(point_pairs)
        lineSet.colors = o3d.utility.Vector3dVector(line_colors)
        center = pcd.get_center()
        vc1.set_lookat(center)
        vc1.set_up([0,1,0])
        vc1.set_zoom(0.8)
        vc1.set_front([0,0,1])
        if frame_idx == 0:
            vis1.add_geometry(pcd)
            vis1.add_geometry(lineSet)
        else:
            vis1.update_geometry(pcd)
            vis1.update_geometry(lineSet)
        vis1.poll_events()
        vis1.update_renderer()
        time.sleep(0.03)

vis1.destroy_window()

del render_option1
del vis1
del vc1

In [12]:
# Static view
pcd = o3d.geometry.PointCloud()

snap_of_motion = all_motion_data['motion'][0][0,:,:,0]

pcd.points = o3d.utility.Vector3dVector(snap_of_motion)
visualization.draw_geometries([pcd],
                              zoom = 1.2,
                              front = [0,0,1],
                              lookat = pcd.get_center(),
                              up = [0,1,0]
                              )

In [13]:
snap_of_motion.shape

(22, 3)